# Threat Intel - Building A Simple Botnet Tracker
> Introduction to threat intel

- toc: true 
- badges: true
- categories: [threatintel,botnet,tracker,automation,python,dbatloader]


![](https://i.imgur.com/poIemWD.jpg)

## Overview
This is a **very simplified** overview of the cyber threat intelligence production cycle and consumers. It is only meant to inform the most casual of reader as to **why we reverse engineer malware**.

### Corporate Security
In a mature corporate security program there needs to be a way for the program to track success, and plan for the future. This is where **threat intel** plays the largest role. The **threat intel** product can both provide a picture of the current and emerging threats faced by an organization, as well as provide operational support for their security controls (data feeds for blocklists etc.)

![](https://i.imgur.com/N4pkf05.png)

### Corporate Consumption of The Intel Product
The corporate value proposition for **threat intel** is simple; *we give you a picture of the threats you face and how they operate so you can protect yourself*. In practice the intel product itself takes many forms each oriented towards a different consumer within the organization. 

![](https://i.imgur.com/MF6vG6r.png)

### Intel Production Process
The **threat intel** production process can be visualized as a funnel with raw data consumed at the opening of the funnel and finished intel product produced at the narrow end. In practice the customer requirements usually drive the finished intelligence product while the internal intel process may drive the raw data collection.

#### Finished Intelligence
With each step in the in intelligence production pipeline the information is refined and enriched to provide a more informed and compelling picture of the current threat landscape. Technical reports produced by **reverse engineers** at the *technical analysis* layer may be producing product that is complete enough to be directly consumed by technical functions within the customer organization.**Intelligence analysts** sit at the narrow end of the intelligence funnel and are not necessarily technical. The final product from an intelligence analyst can often by summarized for briefing at the C-level of the customer organization.

#### Operational Intelligence 
With the emergence of the **detection engineering** role (both within the customer organization as well as within the intel production pipeline) there is also a secondary funnel. The primary role of the detection engineer within the intel pipeline is to produce raw intelligence (one step above data) that can be machine consumable for security controls (rules, IOCs, etc.) The primary role of the detection engineer within the customer organization is to consume this raw intelligence and ensure it is fed into their security controls. This hybrid role forms a synergy (cringe) between the intel production pipeline and the security controls products (EDR, FIREWALL, etc.) Sometimes this secondary funnel is referred to as **operational intelligence**.

![](https://i.imgur.com/uRH0N6k.png)


## Simple DbatLoader Tracker

Our tracker will be responsible for pulling down the payloads deployed by **dbatloader**. Each loader sample contains a download URL which can be used to download a unique (I think?) payload. The payloads are encrypted with a simple format that we reverse engineered on a past [stream](https://research.openanalysis.net/dbatloader/delphi/loader/config/triage/2022/09/04/dbatloader.html).

### The Tracker Architecture 
- Use the UnpacMe feed to pull all new dbatloader ULRs
- Download the payloads
- Decrypt them and ???

**TODO**